Importing Required Librarries


In [ ]:
import os
import cv2
import torch
import shutil
import random
import numpy as np
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt


np.random.seed(42)

Mount google drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git

In [ ]:
%cd yolov5/
!pwd

/content/yolov5
/content/yolov5


In [ ]:
!pip install -r requirements.txt

In [ ]:
import scipy.io

# Load the .mat file
mat = scipy.io.loadmat('/content/Avenue_Dataset/Avenue Dataset/testing_vol/vol01.mat')

# Access the correct key
volLabel = mat['vol']

# Flatten and convert to integer (0 or 1)
gt_labels = volLabel.astype(int).flatten()

print(f"✅ Loaded {len(gt_labels)} ground-truth labels.")
print(gt_labels[:10])  # Print first 10 labels


✅ Loaded 27628800 ground-truth labels.
[42 41 42 41 42 41 42 42 42 42]


Use of pretrained yolo model


In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)
model.conf = 0.25
model.iou = 0.45


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-7-26 Python-3.11.13 torch-2.6.0+cu124 CPU

100%|██████████| 166M/166M [00:01<00:00, 146MB/s]

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients, 205.5 GFLOPs
Adding AutoShape... 


In [ ]:
! pip install deep_sort_realtime opencv-python tqdm

In [ ]:
from deep_sort_realtime.deepsort_tracker import DeepSort

Making of output folder

In [ ]:
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, classification_report
import scipy.io

# --- Load ground truth ---


# Load the .mat file
gt_path = '/content/Avenue_Dataset/Avenue Dataset/testing_vol/vol01.mat'
mat = scipy.io.loadmat(gt_path)

# Access the 'vol' key (which is a 3D array of shape (H, W, N_frames))
vol = mat['vol']  # shape: (H, W, N)

print("vol shape:", vol.shape)  # e.g., (128, 128, 1439)

# Convert to 1 label per frame: 1 if any pixel is anomalous in that frame, else 0
gt_labels = [1 if np.max(vol[:, :, i]) > 0 else 0 for i in range(vol.shape[2])]

print(f"✅ Loaded {len(gt_labels)} frame-level GT labels.")

# --- Your Tracker Setup ---
tracker = DeepSort(max_age=30)
N_history = 10
velocity_threshold = 50  # Set based on experiment
smoothing_frames = 10

# Input/output paths
input_video_path = '/content/Avenue_Dataset/Avenue Dataset/testing_videos/01.avi'  # ✅ CHANGE this
output_video_path = '/content/output_video.avi'

# Read video
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"🎥 Video: {fps} FPS, {width}x{height}, {frame_count} frames")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

read_count = 0
frame_idx = 0
pbar = tqdm(total=frame_count)

track_memory = {}
anomaly_memory = {}
predicted_labels = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()

    formatted_detections = []
    for d in detections:
        xmin, ymin, xmax, ymax, conf, cls = d
        if conf < 0.5:
            continue  # skip low-confidence
        if int(cls) != 0:
            continue  # only track 'person'
        box = [xmin, ymin, xmax - xmin, ymax - ymin]
        formatted_detections.append([box, conf])


    tracks = tracker.update_tracks(formatted_detections, frame=frame)
    frame_anomaly = False

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        xmin, ymin, xmax, ymax = map(int, ltrb)
        center_x = (xmin + xmax) / 2
        center_y = (ymin + ymax) / 2

        track_memory.setdefault(track_id, []).append((frame_idx, center_x, center_y))
        track_memory[track_id] = track_memory[track_id][-N_history:]

        if len(track_memory[track_id]) >= 2:
            f1, x1, y1 = track_memory[track_id][0]
            f2, x2, y2 = track_memory[track_id][-1]
            dt = (f2 - f1) / fps
            dx = x2 - x1
            dy = y2 - y1
            distance = np.sqrt(dx ** 2 + dy ** 2)
            velocity = distance / (dt + 1e-6)
        # Normalize by frame size
            norm_velocity = velocity / np.sqrt(width**2 + height**2)

            # Threshold (tunable)
            anomaly = norm_velocity > 0.05
        else:
            anomaly = False

        if anomaly:
            anomaly_memory[track_id] = frame_idx

        if track_id in anomaly_memory:
            if frame_idx - anomaly_memory[track_id] <= smoothing_frames:
                anomaly = True

        if anomaly:
            frame_anomaly = True
            label = f"ID {track_id} ANOMALY"
            color = (0, 0, 255)
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 1)
            cv2.putText(frame, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    predicted_labels.append(1 if frame_anomaly else 0)

    out.write(frame)
    frame_idx += 1
    read_count += 1
    pbar.update(1)

pbar.close()
cap.release()
out.release()

print(f"✅ Frames processed: {read_count} / {frame_count}")
print(f"🎬 Output saved to: {output_video_path}")


vol shape: (120, 160, 1439)
✅ Loaded 1439 frame-level GT labels.
🎥 Video: 25.0 FPS, 640x360, 1439 frames


  0%|          | 0/1439 [00:00<?, ?it/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
  0%|          | 1/1439 [00:06<2:37:20,  6.57s/it]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
  0%|          | 2/1439 [00:09<1:44:06,  4.35s/it]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
  0%|          | 3/1439 [00:12<1:35:42,  4.00s/it]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated.

✅ Frames processed: 1439 / 1439
🎬 Output saved to: /content/output_video.avi


In [ ]:
print("Ground Truth Label Distribution:", np.unique(gt_labels, return_counts=True))


In [ ]:
# After video processing is done
predicted_labels = np.array(predicted_labels)

# Load ground truth
mat = scipy.io.loadmat('/content/Avenue_Dataset/Avenue Dataset/testing_vol/vol01.mat')
anomaly_frames = mat['vol'].flatten()

# Create GT labels
gt_labels = np.zeros(len(predicted_labels), dtype=int)
gt_labels[anomaly_frames - 1] = 1

# Evaluate
if len(predicted_labels) != len(gt_labels):
    print(f"⚠️ Warning: label lengths differ — predicted={len(predicted_labels)}, GT={len(gt_labels)}")
else:
    auc = roc_auc_score(gt_labels, predicted_labels)
    print(f"🎯 AUC Score: {auc:.4f}")
    print("\nClassification Report:")
    print(classification_report(gt_labels, predicted_labels > 0.5, digits=4))


🎯 AUC Score: 0.5013

Classification Report:
              precision    recall  f1-score   support

           0     1.0000    0.0025    0.0050      1189
           1     0.1741    1.0000    0.2966       250

    accuracy                         0.1758      1439
   macro avg     0.5870    0.5013    0.1508      1439
weighted avg     0.8565    0.1758    0.0557      1439

